# Régression Linéraire pour la prédiction des prix de voiture

Modèle simple pour prédire les prix en fonction du kilométrage.

## Définitions :

- **Normalisation** : Redimensionner les variables numériques pour qu'elles soient comparables sur une échelle commune.
- **Machine Learning** : Donner à une machine la capacité d'apprendre sans la programmer de façon explicite.
- **Apprentissage supervisé** : Technique d'apprentissage la plus courante en machine learning. On donne des exemples à la machine qu'elle doit étudier pour en créer des modèles.
- **Dataset** : Ensemble d'exemples données à la machine pour apprendre (tableau de données).
- **Problème de regression** : On cherche à prédire la valeur d'une variable continue, c'est-à-dire une variable qui peux prendre une infinité de valeurs.
- **Problème de classification** : On cherche à prédire la valeur d'une variable discrète, c'est-à-dire une variable qui prends certaines valeurs.

## Étape 1 : Charger les données

Fonction permettant de lire et d'extraire des données d'un fichier CSV.

In [ ]:
from csv import reader

def read_csv(self) -> tuple[list[list[float]], list[list[float]]]:
    mileage: list[list[float]] = []
    price: list[list[float]] = []

    try:
        with open(self.path, mode='r', newline='') as file:
            r = reader(file)
            next(r)
            for row in r:
                mileage.append([float(row[0])])
                price.append([float(row[1])])

        return mileage, price
    except FileNotFoundError:
        raise RuntimeError(f"The file {self.path} does not exist.")
    except Exception as e:
        raise RuntimeError(f"An error occurred while reading the CSV file: {e}")

Représentation visuel des donnés brutes contenu dans le fichier CSV.

![Raw Data Visualisation](images/raw_data.png)

Par convention, on note `m` le nombre d'exemple que l'on a dans notre dataset.
On note `n` le nombre de features que possède notre dataset (c'est-à-dire le nombre de colonne hormis la colonne target).

![Dataset representation](images/dataset.png)

Dans notre situation, nous aurons donc un vecteur target (prix) de taille `(m, 1)` et une matrice features (km) de taille `(m, 1)` où `m` est égal à 24.

Dans notre situation, nous avons :

$$
\vec{y}=\begin{pmatrix}240000\cr139800\cr\ldots\cr61789\cr\end{pmatrix}
$$
$$
\vec{x}=\begin{pmatrix}3650\cr3800\cr\ldots\cr8290\cr\end{pmatrix}
$$

## Étape 2 : Normaliser les données du fichier CSV

Fonction permettant de normalier les données extrait du dataset.

La formule permettant de normaliser des données est :
$$
x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}
$$


In [6]:
def normalize(self, matrix: list[list[float]]) -> list[list[float]]:
    xmin: float = self.__matrix_min(matrix) # Recherche de la valeur minimum dans la matrice.
    xmax: float = self.__matrix_max(matrix) # Recherche de la valeur maximale dans la matrice.
    return [[(item[0] - xmin) / (xmax - xmin)] for item in matrix]

Représentation visuel des donnés normalisés (représentant le kilométrage).
![Normalized Mileages](images/normalize_mileages.png)